In [27]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
#from sklearn.feature_extraction import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import mean_squared_error

In [ ]:
/root/hackerday/13_product_search/train.csv.zip
/root/hackerday/13_product_search/test.csv.zip
/root/hackerday/13_product_search/sample_submission.csv.zip
/root/hackerday/13_product_search/product_descriptions.csv.zip
/root/hackerday/13_product_search/attributes.csv.zip

In [28]:
import zipfile

In [29]:
zf = zipfile.ZipFile('/root/hackerday/13_product_search/train.csv.zip')
df_train = pd.read_csv(zf.open('train.csv'),encoding="ISO-8859-1")

In [30]:
zf = zipfile.ZipFile('/root/hackerday/13_product_search/test.csv.zip')
df_test = pd.read_csv(zf.open('test.csv'),encoding="ISO-8859-1")

In [37]:
zf = zipfile.ZipFile('/root/hackerday/13_product_search/product_descriptions.csv.zip')
df_pro_desc = pd.read_csv(zf.open('product_descriptions.csv'),encoding="ISO-8859-1")

In [12]:
train.head()

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67


In [33]:
num_train = df_train.shape[0]
num_test = df_test.shape[0]

In [26]:
df_test.head()

,id,product_uid,product_title,search_term
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able
3,6,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong ties
4,7,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong tie hcc668


In [40]:
df_pro_desc.head()
#df_pro_desc.shape[0]

,product_uid,product_description
0,100001,"Not only do angles make joints stronger, they ..."
1,100002,BEHR Premium Textured DECKOVER is an innovativ...
2,100003,Classic architecture meets contemporary design...
3,100004,The Grape Solar 265-Watt Polycrystalline PV So...
4,100005,Update your bathroom with the Delta Vero Singl...


In [17]:
def str_stemmer(s):
    return " ".join([stemmer.stem(word) for word in s.lower().split()])

In [18]:
def str_common_word(str1,str2):
    return sum(int(str2.find(word)>=0) for word in str1.split())

In [34]:
df_all = pd.concat((df_train,df_test),axis=0,ignore_index=True)

In [36]:
df_all.head()
df_all.shape[0]

240760

In [42]:
df_all = pd.merge(df_all,df_pro_desc,how='left',on='product_uid')

In [44]:
df_all.head(10)

,id,product_title,product_uid,relevance,search_term,product_description
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket,"Not only do angles make joints stronger, they ..."
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,"Not only do angles make joints stronger, they ..."
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,BEHR Premium Textured DECKOVER is an innovativ...
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,Update your bathroom with the Delta Vero Singl...
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet,Update your bathroom with the Delta Vero Singl...
5,18,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,3.00,convection otr,Achieving delicious results is almost effortle...
6,20,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,2.67,microwave over stove,Achieving delicious results is almost effortle...
7,21,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,3.00,microwaves,Achieving delicious results is almost effortle...
8,23,Lithonia Lighting Quantum 2-Light Black LED Em...,100007,2.67,emergency light,The Quantum Adjustable 2-Light LED Black Emerg...
9,27,House of Fara 3/4 in. x 3 in. x 8 ft. MDF Flut...,100009,3.00,mdf 3/4,Get the House of Fara 3/4 in. x 3 in. x 8 ft. ...


In [45]:
from nltk.stem.snowball import SnowballStemmer

In [46]:
stemmer = SnowballStemmer('english')

In [57]:
import nltk
from nltk.tokenize import word_tokenize

In [ ]:
#IDF(light)= 1 + log(total number of documents/number of documents having light)

In [ ]:
#Tf*IDF multiplication and matrix
#vector space model

In [61]:
#d1- search term
#d2- description 1
# cos(d1,d2) = (d1.d2)/||d1||.||d2||
# d1 and d2 indicates vector dot product
# ||d1|| the length of the vector

d1= (5,0,3,0,2,0,0,2,0,0)
d2= (3,0,2,0,1,1,0,1,0,1)

d1.d2 = 5*3+0*0+ =   34

||d1|| = (5*5+0*0+3*3) = (45)^0.5 = 6.7
||d2|| = (3*3+0*0+2*2) = (18)^0.5 = 4.24
#

In [65]:
#cos(d1,d2) = 34 / 6.7*4.24 = 1.19

In [70]:
#porters stemmer
#lancaster stemmer
#snowball stemmer

df_all['search_term'] = df_all['search_term'].map(lambda x:str_stemmer(x))

In [73]:
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stemmer(x))

In [71]:
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stemmer(x))


In [68]:
df_all.head(10)

,id,product_title,product_uid,relevance,search_term,product_description
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angl bracket,"Not only do angles make joints stronger, they ..."
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,"Not only do angles make joints stronger, they ..."
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,BEHR Premium Textured DECKOVER is an innovativ...
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,Update your bathroom with the Delta Vero Singl...
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower onli faucet,Update your bathroom with the Delta Vero Singl...
5,18,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,3.00,convect otr,Achieving delicious results is almost effortle...
6,20,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,2.67,microwav over stove,Achieving delicious results is almost effortle...
7,21,Whirlpool 1.9 cu. ft. Over the Range Convectio...,100006,3.00,microwav,Achieving delicious results is almost effortle...
8,23,Lithonia Lighting Quantum 2-Light Black LED Em...,100007,2.67,emerg light,The Quantum Adjustable 2-Light LED Black Emerg...
9,27,House of Fara 3/4 in. x 3 in. x 8 ft. MDF Flut...,100009,3.00,mdf 3/4,Get the House of Fara 3/4 in. x 3 in. x 8 ft. ...


In [72]:
df_all.head()

,id,product_title,product_uid,relevance,search_term,product_description
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angl bracket,"not onli do angl make joint stronger, they als..."
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,"not onli do angl make joint stronger, they als..."
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,behr premium textur deckov is an innov solid c...
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,updat your bathroom with the delta vero single...
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower on faucet,updat your bathroom with the delta vero single...


In [74]:
df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)


In [ ]:
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title']+"\t"+df_all['product_description']


In [ ]:
df_all['word_in_title']=df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))


In [ ]:
df_all['word_in_description']=df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))


In [ ]:
df_all = df_all.drop(['search_term','product_title','product_description','product_info'],axis=1)


In [ ]:
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']

In [ ]:
y_train = df_train['relevance'].values
X_train = df_train.drop(['id','relevance'],axis=1).values
X_test = df_test.drop(['id','relevance'],axis=1).values

In [ ]:
rf = RandomForestRegressor(n_estimators=15, max_depth=6,random_state=0)
clf = BaggingRegressor(rf,n_estimators=45,max_samples=0.1,random_state=25)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)


In [ ]:
pd.DataFrame({'id':id_test,'pred_relevance':y_pred})